In [ ]:
import boto3
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [ ]:
session = boto3.session.Session()
ec2 = session.client("ec2")

In [ ]:
def available_zones(ec2):
    response = ec2.describe_availability_zones()
    availability_zones = response["AvailabilityZones"]
    available_zones = [
        zone for zone in availability_zones if zone["State"] == "available"
    ]
    return [name["ZoneName"] for name in available_zones]


def gpu_instance_types(ec2):
    paginator = ec2.get_paginator(
        ec2.get_instance_types_from_instance_requirements.__name__
    )
    page_iterator = paginator.paginate(
        ArchitectureTypes=["x86_64"],
        VirtualizationTypes=["hvm"],
        InstanceRequirements={
            "VCpuCount": {"Min": 1},
            "MemoryMiB": {"Min": 1},
            "InstanceGenerations": ["current"],
            "SpotMaxPricePercentageOverLowestPrice": 100,
            "OnDemandMaxPricePercentageOverLowestPrice": 20,
            "AcceleratorTypes": ["gpu"],
            "AcceleratorCount": {"Min": 1},
            "AcceleratorManufacturers": ["nvidia"],
            "AcceleratorNames": ["a100", "v100", "k80", "t4", "m60"],
        },
    )
    return [
        instance_type["InstanceType"]
        for page in page_iterator
        for instance_type in page["InstanceTypes"]
    ]


def spot_supported_instance_types(ec2, instance_types=[]):
    paginator = ec2.get_paginator(ec2.describe_instance_types.__name__)
    page_iterator = paginator.paginate(
        InstanceTypes=instance_types,
        Filters=[{"Name": "supported-usage-class", "Values": ["spot"]}],
    )
    return [
        instance_type["InstanceType"]
        for page in page_iterator
        for instance_type in page["InstanceTypes"]
    ]

# EC2 available locations

In [ ]:
regions_dropdown = widgets.Dropdown(
    options=session.get_available_regions("ec2"),
    value=session.region_name,
    description="Region:",
)
regions_dropdown

In [ ]:
# Note that ec2 client is reassigned
ec2 = session.client("ec2", region_name=regions_dropdown.value)
zones_dropdown = widgets.Dropdown(
    options=available_zones(ec2),
    description="Zone:",
)
zones_dropdown

# Instance type offerings

See [get_instance_types_from_instance_requirements](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/ec2.html#EC2.Client.get_instance_types_from_instance_requirements), [Recommended GPU Instances](https://docs.aws.amazon.com/dlami/latest/devguide/gpu.html), and [Amazon EC2 Instance Types](https://aws.amazon.com/ec2/instance-types/) for more on instances and GPU offerings

In [ ]:
types = spot_supported_instance_types(ec2, gpu_instance_types(ec2))
types_dropdown = widgets.Dropdown(
    options=types,
    value="p2.xlarge" if "p2.xlarge" in types else types[0],
    description="Type:",
)
types_dropdown

# Pricing

In [ ]:
response = ec2.describe_spot_price_history(
    AvailabilityZone=zones_dropdown.value,
    InstanceTypes=[types_dropdown.value],
    MaxResults=50,
)

In [ ]:
spot_price_history = response["SpotPriceHistory"]
prices = [float(item["SpotPrice"]) for item in spot_price_history]
time_stamps = [item["Timestamp"].isoformat() for item in spot_price_history]

In [ ]:
plt.xticks(rotation=90)
plt.plot(time_stamps, prices)